<img height="40px" src="https://science-tokyo.box.com/shared/static/d3p6jeu3bwsaid6xu1k56q05luvmkbd4.png">

In [ ]:
# Preparation for auto-grading
unit = "suppl_EFDS6"
!pip install otter-grader 
!wget https://github.com/keisuke-yanagisawa/isct-efdsai/raw/refs/heads/main/supplementary_materials/{unit}/student.tar.gz -O {unit}.tar.gz
!tar -zxvf {unit}.tar.gz 
!rm {unit}.tar.gz
%cd student
import otter
grader = otter.Notebook()

# 基盤データサイエンス演習 第6回 次元削減 補習資料

# Supplemental Materials: Exercises in Fundamentals of Data Science (6) - Dimensionality Reduction

## 1 | Moonデータセットを使った次元削減手法の比較

## 1 | Comparison of Dimensionality Reduction Methods Using the Moon Dataset

#### 事前準備

#### Preparations

In [ ]:
from sklearn import datasets
import matplotlib.pyplot as plt

q1_X, q1_y = datasets.make_moons(100, noise=0.05, random_state=42)
plt.scatter(q1_X[q1_y==0, 0], q1_X[q1_y==0, 1])
plt.scatter(q1_X[q1_y==1, 0], q1_X[q1_y==1, 1])

#### 練習問題 1-1 | PCAによる次元削減

#### Exercise 1-1 | Dimensionality Reduction with PCA

`q1_X` に対してPCAを実施し、第1主成分得点のみを取り出したものを `q11_pc1_score` に記録せよ。
また、`q11_pc1_score` について、`q1_y==0` であるデータと `q1_y==1` であるデータとでどの程度値に差があるのか、ヒストグラムを用いて評価せよ。（後半は自動採点対象外である）

Perform PCA on `q1_X` and record only the first principal component score in `q11_pc1_score`.
Also, evaluate the extent to which there is a difference in values between data where `q1_y==0` and `q1_y==1`, using a histogram for assessment. (The latter part is not subject to auto-grading.)

In [ ]:
from sklearn.decomposition import PCA
...

In [ ]:
q11_pc1_score_0 = q11_pc1_score[q1_y==0]
q11_pc1_score_1 = q11_pc1_score[q1_y==1]
plt.hist(q11_pc1_score_0, label="label=0", alpha=0.5)
plt.hist(q11_pc1_score_1, label="label=1", alpha=0.5)
plt.ylabel("PC1 score")
plt.legend()
plt.show()

In [ ]:
grader.check("Exercise 1-1")

#### 練習問題 1-2 | Graph Embeddingによる次元削減

#### Exercise 1-2 | Dimensionality Reduction with Graph Embedding

`q1_X` に対して `SpectralEmbedding` を実施し、第1主成分得点のみを取り出したものを `q12_pc1_score` に記録せよ。ただし、 `affinity="Nearest_neighbors"` とし、 `n_neighbors=10` とせよ。
また、`q12_pc1_score` について、`q1_y==0` であるデータと `q1_y==1` であるデータとでどの程度値に差があるのか、boxplotを用いて評価せよ。（後半は自動採点対象外である）

Perform `SpectralEmbedding` on `q1_X` and record only the first principal component score in `q12_pc1_score`. Use `affinity="Nearest_neighbors"` and set `n_neighbors=10`.
Also, evaluate the extent to which there is a difference in values between data where `q1_y==0` and `q1_y==1`, using a boxplot for assessment. (The latter part is not subject to auto-grading.)

In [ ]:
from sklearn.manifold import SpectralEmbedding
...

In [ ]:
q12_pc1_score_0 = q12_pc1_score[q1_y==0]
q12_pc1_score_1 = q12_pc1_score[q1_y==1]
plt.hist(q12_pc1_score_0, label="label=0", alpha=0.5)
plt.hist(q12_pc1_score_1, label="label=1", alpha=0.5)
plt.ylabel("PC1 score")
plt.legend()
plt.show()

In [ ]:
grader.check("Exercise 1-2")

#### 練習問題 1-3 | PCAの計算時間とGEの計算時間

#### Exercise 1-3 | Comparison of Calculation Times for PCA and GE

以下のコードを実行し、3000件のデータに対する計算時間を比較せよ。この練習問題は自動採点対象外である。

Run the following code and compare the computation times for 3000 data points. This exercise is not subject to auto-grading.

In [ ]:
q13_X, q13_y = datasets.make_moons(3000, noise=0.05, random_state=42)

In [ ]:
# PCA

import time

st = time.time()

ge = PCA(n_components=2)
ge.fit_transform(q13_X)

ed = time.time()
print(f"{ed-st:.2f} sec.")

In [ ]:
# Graph Embedding

import time

st = time.time()

ge = SpectralEmbedding(n_components=2, affinity="nearest_neighbors", n_neighbors=100)
ge.fit_transform(q13_X)

ed = time.time()
print(f"{ed-st:.2f} sec.")

## 2 | CCAの低次元空間におけるデータ間の距離

## 2 | Distances Between Data in Low-Dimensional Space with CCA

#### 事前準備

#### Preliminaries

In [ ]:
from sklearn import datasets
from sklearn.model_selection import train_test_split

q2_d = datasets.load_digits()
q2_D = q2_d.data
q2_y = q2_d.target

q2_D_subset, _, q2_y_subset, _ = train_test_split(q2_D, q2_y, train_size=300, random_state=42) # extract only 300 samples
q2_X_subset = q2_D_subset[:, :48] # upper side of the digit image
q2_Z_subset = q2_D_subset[:, 48:] # lower side of the digit image

In [ ]:
## Function for 2D visualization of handwritten character data
import matplotlib.pyplot as plt
def plot_digits(X, y):
    plt.figure(figsize=(5,5))
    color = [plt.cm.nipy_spectral(i/10, 1) for i in range(10)]
    for i in range(10):
        plt.scatter(X[y==i, 0], X[y==i, 1], color=color[i], marker=f"${i}$")
    plt.show()

In [ ]:
# Perform CCA between X_subset and Z_subset
from sklearn.cross_decomposition import CCA

q2_cca = CCA(n_components=2, scale=True)
q2_cca.fit(q2_X_subset, q2_Z_subset)
q2_Xc, q2_Zc = q2_cca.transform(q2_X_subset, q2_Z_subset)

In [ ]:
# please uncomment below lines if you want to see the plot

# plot_digits(q2_Xc, q2_y_subset) 
# plot_digits(q2_Zc, q2_y_subset)

#### 練習問題 2-1 | データ間の距離を求める関数

#### Exercise 2-1 | Function to Calculate Distances Between Data

CCAでは、2つのデータ間の距離を縮めるように低次元空間への射影を行う。したがって、得られた低次元表現 $X_c$ および $Z_c$ は、対応関係のあるデータ間の距離が近いことが期待される。

そこで、データ間の距離を求める関数 `q21_calc_dist(v1, v2)` を実装せよ。ただし、`v1` および `v2` はそれぞれベクトルであり、ユークリッド距離を用いて距離を求めるものとする。

CCA projects two sets of data into a lower dimensional space in order to reduce the distance between them. Therefore, the obtained low-dimensional representations $X_c$ and $Z_c$ are expected to have close distances for corresponding data.

Implement the function `q21_calc_dist(v1, v2)` to calculate the distance between data, where `v1` and `v2` are vectors, and the Euclidean distance is used to calculate the distance.

In [ ]:
import numpy as np

def q21_calc_dist(v1, v2):
    ...
    return ...

In [ ]:
grader.check("Exercise 2-1")

#### 練習問題 2-2 | 対応しあうデータ間の距離

#### Exercise 2-2 | Distances Between Corresponding Data

練習問題 2-1 で作成した `q21_calc_dist` 関数を用いて、`q2_Xc` と `q2_Zc` の対応しあう各データ間の距離を求め、その平均値を `q22_avg_dist_paired` に記録せよ。 

Using the `q21_calc_dist` function created in Exercise 2-1, calculate the distances between corresponding data in `q2_Xc` and `q2_Zc`, and record the average value in `q22_avg_dist_paired`.

In [ ]:
...
q22_avg_dist_paired = ...

In [ ]:
grader.check("Exercise 2-2")

#### 練習問題 2-3 | 対応しないデータ間の距離との比較

#### Exercise 2-3 | Comparison with Distances Between Unpaired Data

`q2_Xc` と `q2_Zc` の**対応しあわない**各データ間の距離を求め（300サンプル存在することから、$300^2 - 300 = 89700$ 件の距離を求めることになるはずである）、その平均値を `q23_avg_dist_unpaired` に記録せよ。 この値を `q22_avg_dist_paired` と比較し、距離の差があることを確認せよ。

Calculate the distances between **unpaired** data in `q2_Xc` and `q2_Zc` (there should be $300^2 - 300 = 89700$ distances calculated), and record the average value in `q23_avg_dist_unpaired`. Compare this value with `q22_avg_dist_paired` to confirm the difference in distances.

In [ ]:
...
q23_avg_dist_unpaired = ...

In [ ]:
grader.check("Exercise 2-3")

## 3 | CCAによる画像復元の復元精度評価

## 3 | Evaluation of Restoration Accuracy of Images Using CCA

#### 事前準備

#### Preliminaries


In [ ]:
q3_cca = CCA(n_components=8, scale=True)
q3_cca.fit(q2_X_subset, q2_Z_subset)
q3_Xc, q3_Zc = q3_cca.transform(q2_X_subset, q2_Z_subset)
q3_X_reconstructed = q3_cca.inverse_transform(q3_Xc)

#### 練習問題 3-1 | cos類似度：ベクトル間の類似度指標

#### Exercise 3-1 | Cosine Similarity: A Similarity Measure Between Vectors

ベクトル間の類似度を測る指標として、cos類似度がよく用いられる。cos類似度は、2つのベクトル $v_1$ および $v_2$ の内積を用いて以下のように定義される。

Cosine similarity is commonly used as a measure of similarity between two vectors. It is defined as follows using the dot product of the two vectors $v_1$ and $v_2$.

$$
\text{cosine similarity} = \frac{v_1 \cdot v_2}{\|v_1\| \|v_2\|}
$$

cos類似度を求める関数 `q31_cos_similarity(v1, v2)` を実装せよ。ただし、`v1`および`v2`は零ベクトルではないものとする。


Implement the function `q31_cos_similarity(v1, v2)` to calculate cosine similarity, where `v1` and `v2` are not zero vectors.


In [ ]:
import numpy as np
def q31_cos_similarity(v1, v2):
    ...
    return ...

In [ ]:
grader.check("Exercise 3-1")

#### 練習問題 3-2 | 自分自身の復元精度の評価

#### Exercise 3-2 | Evaluation of Self-Reconstruction Accuracy

CCAを用いて `q2_X_subset` を一旦低次元空間に射影し、元の空間に復元したものを `q3_X_reconstructed` として記録してある（事前準備のコードを確認せよ）。変換前と変換後の各データ間のcos類似度を求め、その平均値を `q32_avg_cos_similarity` に記録せよ。

Using CCA, project `q2_X_subset` into a low-dimensional space and record the reconstruction in the original space as `q3_X_reconstructed` (check the code in the preparations). Calculate the cosine similarity between the original and reconstructed data for each data point, and record the average value in `q32_avg_cos_similarity`.

In [ ]:
...
q32_avg_cos_similarity = ...

In [ ]:
grader.check("Exercise 3-2")

#### 練習問題 3-3 | 最も復元しやすい数字の特定

#### Exercise 3-3 | Identifying the Most Easily Restorable Digit

cos類似度の平均値が高い数字は復元が容易な数字であると推定出来る。最もcos類似度の平均値が高い数字を特定し、その数字を `q33_best_digit` に整数値として記録せよ（例えば文字`"5"`ではなく、整数値`5`として記録せよ）。

Digits with high average cosine similarity are presumed to be easily restorable. Identify the digit with the highest average cosine similarity and record that digit as an integer in `q33_best_digit` (record as an integer value like `5` instead of the character `"5"`).

In [ ]:
...
q33_best_digit ...

In [ ]:
grader.check("Exercise 3-3")